# Table Question Generation: Inference example

In this notebook, we will show how to use a pretrained TableQG model to generate questions. We first sample SQL queries from a given table, and then use a text-to-text transformer (T5) to transcribe the SQL query to a natural language question. 

## Dependencies

If not already done, make sure to install OneQA with notebooks extras before getting started.

In [1]:
from primeqa.qg.models.qg_model import QGModel
from tabulate import tabulate # only used to visualize table

## Loading pretrained model from huggingface

This model was trained using OneQA library and uploaded to huggingface.

In [2]:
model_name = 'ibm/t5-base-table-question-generator'
table_qg_model = QGModel(model_name, modality='table')

<br>

## Sample table

Tables should be passed a `list` of `dicts`. Each `dict` will correspond to a table with keys `"header"` and `"rows"`.

In [3]:
table_list = [
    {"header": ["Player", "No.", "Nationality", "Position", "Years in Toronto", "School Team"],
      "rows": [
            ["Antonio Lang", 21, "United States", "Guard-Forward", "1999-2000", "Duke"],
            ["Voshon Lenard", 2, "United States", "Guard", "2002-03", "Minnesota"],
            ["Martin Lewis", 32, "United States", "Guard-Forward", "1996-97", "Butler CC (KS)"],
            ["Brad Lohaus", 33, "United States", "Forward-Center", "1996", "Iowa"],
            ["Art Long", 42, "United States", "Forward-Center", "2002-03", "Cincinnati"]
        ]
    }
]

print(tabulate(table_list[0]['rows'], headers=table_list[0]['header'], tablefmt='grid'))

+---------------+-------+---------------+----------------+--------------------+----------------+
| Player        |   No. | Nationality   | Position       | Years in Toronto   | School Team    |
+===============+=======+===============+================+====================+================+
| Antonio Lang  |    21 | United States | Guard-Forward  | 1999-2000          | Duke           |
+---------------+-------+---------------+----------------+--------------------+----------------+
| Voshon Lenard |     2 | United States | Guard          | 2002-03            | Minnesota      |
+---------------+-------+---------------+----------------+--------------------+----------------+
| Martin Lewis  |    32 | United States | Guard-Forward  | 1996-97            | Butler CC (KS) |
+---------------+-------+---------------+----------------+--------------------+----------------+
| Brad Lohaus   |    33 | United States | Forward-Center | 1996               | Iowa           |
+---------------+-------+-----

## Generate questions

There are some arguments to control the type of questions generated.
#### Controls:
- `num_questions_per_instance`: Number of questions to generate per table (default=5)
- `agg_prob`: Probability distribution over aggregates. `agg_prob` should be a probability vector of length 6, with each index giving prob of an aggregate ops appearing in this order `['select', 'maximum', 'minimum', 'count', 'sum', 'average']`. (default=`[1,0,0,0,0,0]`)
- `num_where_prob`: Should be a vector of size 5 with probablities of number of where clauses to use while generating sqls. If k where clause can't be generated the code tries to generate k-1 where clause query, and so on. (default: [0,1,0,0,0])
- `ineq_prob`: Probability of generating inequality clauses in SQLs. It should be a float.(default=0.0)

In [4]:
table_qg_model.generate_questions(table_list, 
                                    num_questions_per_instance = 10,
                                    agg_prob = [1.,0,0,0,0,0],
                                    num_where_prob = [0,1.,0,0,0],
                                    ineq_prob = 0.0)

[{'question': 'What position does the player from Butler CC (KS) play?',
  'answer': 'guard-forward'},
 {'question': 'Which Position has a No. of 33.0?', 'answer': 'forward-center'},
 {'question': 'What school team did Antonio Lang play for?', 'answer': 'duke'},
 {'question': 'Name the years in toronto for number 32.0',
  'answer': '1996-97'},
 {'question': 'Name the years in toronto for iowa', 'answer': '1996'},
 {'question': 'What school team did Voshon Lenard play for?',
  'answer': 'minnesota'},
 {'question': 'What is the nationality of the player from Cincinnati?',
  'answer': 'united states'},
 {'question': 'Which player played in Toronto in 1996-97?',
  'answer': 'martin lewis'},
 {'question': 'Which School Team has a No. of 2.0?', 'answer': 'minnesota'},
 {'question': 'Which player has a number of 33.0?', 'answer': 'brad lohaus'}]